# Quantum process tomography

In quantum computing process tomography is mostly used to experimentally measure the quality of gates. The quality is how close the experimental implementation of the gate is to the ideal gate.

### Classical "process" tomography 

On a single bit there are two processes (or gates or operations) one can perform, `identity`  and `not`, which we will denote as $I$ and $X$. The `identity` operation behaves as
$I 0 = 0$ and $I 1 = 1$ while `not` behaves as $X 0 = 1$ and $X 1 = 0$, these are usually called truth tables. Thus to do classical process tomography we simply need to input a $0$ and $1$ into the circuit and compare the output to the truth table.

However if logic gates can be faulty the situation becomes more complicated. The exact operation of the faulty gate can be captured by the probability of all output strings $j$ given all possible input strings $i$, i.e. $\Pr({\rm output\ } j| {\rm input\ }i)$. It is convenient to collect these probabilites into a matrix which is called a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix). For a gate $G$ on single bit $ i,j \in \{0, 1 \}$ it is

$$
C(G) =
\begin{pmatrix}
\Pr(0|0) &  \Pr(0|1)\\
\Pr(1|0) &  \Pr(1|1)
\end{pmatrix}.
$$
The ideal confusion matrix for `identity` is $ \Pr( j | i ) = \delta_{i,j} $ while for `not` it is  $ \Pr( j | i ) = 1 -\delta_{i,j} $.

### Quantum process tomography of a single qubit

To motivate the additional complexity of quantum process tomography over classical process tomography consider the following.

We would like to distinguish the following quantum processes
$$
H = \frac{1}{2}
\begin{pmatrix}
1 &  1\\
1 &  -1
\end{pmatrix}.
$$
and 
$$
R_Y(\pi/2) = \exp[-i (\pi/2) Y/2].
$$


To distingiush these processes we estimate the confusion matrices in the standard basis (the Z basis). After many trials the probabilities obey 

$$
\Pr(j|G, \rho_i) = {\rm Tr}[G\rho_i G^\dagger \Pi_j],
$$ 

where $\Pi_j= |j\rangle \langle j|$ is a measurement operator with $j\in\{ 0,1 \}$, $\rho_i=|i\rangle \langle i|$ is the input state with $i\in\{ 0,1 \}$,  and $G$ is the quantum process ie $H$ or $R_Y$. 

Using the expression for $\Pr(j|G, \rho_i)$ we can construct the confusion matrices for each process. Unfortunately confusion matrices are identical
$$
C(H)= C\big(R_Y(\pi/2)\big) = \frac 1 2
\begin{pmatrix}
1 &  1\\
1 &  1
\end{pmatrix}.
$$
However, if we input the states $\rho_+=|+\rangle\langle +|$ and $\rho_-=|-\rangle\langle -|$ the confusion matrices become 
$$
C(H) = \frac 1 2
\begin{pmatrix}
1 & 0\\
0 &  1
\end{pmatrix} 
\quad {\rm and} \quad
C\big(R_Y(\pi/2)\big) = \frac 1 2
\begin{pmatrix}
1 &  1\\
1 &  1
\end{pmatrix}. 
$$
Instead of using a different input state we could have measured in different bases. A rough way to think about quantum process tomography is you need to input a tomographically complete set of input states and measure the confusion matrix of those states in a tomographically complete basis.

A *tomographically complete* set of operators is an operator basis on the Hilbert space of the system. For a single qubit this is the Pauli operators $\{ I, X, Y, Z \}$.


### Quantum process tomography in general   

The above analogy gets further stretched when we consider imperfect gates (non unitary processes). Indeed understanding quantum process tomography is beyond the scope of this notebook.


Quantum process tomography involves
 - preparing a state
 - executing a process (the thing you are trying to estimate)
 - measuring in a basis
 
 <figure>
  <img src="figs/process-tomo.png" alt="Drawing" style="width: 460px;"/>
  <figcaption>Figure 1. For process tomography, rotations must be prepended and appended to fully resolve the action of V on arbitrary initial states.</figcaption>
</figure>
 
 
The process is kept fixed, while the experimental settings (the preparation and measurement) are varied using pre and post rotations, see Figure 1. To estimate a quantum process matrix on $n$ qubits requires estimating $D^4-D^2$ parameters where $D=2^n$ is the dimension of the Hilbert space.



Programmatically ((prep, measure) tuples) are varied. You first choose a suitable set of input states and measurement operators, and then run every `itertools.product` combination of settings.

There are two choices of tomographically complete input states, *SIC* states and *Pauli* states.

The SIC states are the states corresponding to the directions of a [SIC POVM](https://en.wikipedia.org/wiki/SIC-POVM). In this case there are only four states, but we still have to measure in the Pauli basis. The scaling of the number of experiments with respect to number of qubits is therefore $4^n 3^n$.

The alternative is to use $\pm$ eigenstates of the Pauli operators as our tomographically complete input states. In this case there are six input states, and we still have to measure in the Pauli basis. The scaling of the number of experiments with respect to number of qubits is therefore $6^n 3^n$.

**More information** 


When thinking about process tomography it is necessary to understand superoperators. For more information see [superoperator_representations.md](../.././forest-benchmarking/docs/superoperator_representations.md) and the [superoperator_tools ipython notebook](superoperator_tools.ipynb).

Also see the following references:

[CWPHD] *Initialization and characterization of open quantum systems*  
Christopher Wood,  
Chapter 3, PhD Thesis, University of Waterloo (2015)  
http://hdl.handle.net/10012/9557  


[IGST]  *Introduction to Quantum Gate Set Tomography*  
Daniel Greenbaum,  
arXiv:1509.02921 (2015)    
https://arxiv.org/abs/1509.02921  


[PBT]  *Practical Bayesian Tomography*  
Christopher Granade et al.  
New J. Phys. 18, 033024 (2016)  
https://dx.doi.org/10.1088/1367-2630/18/3/033024  
https://arxiv.org/abs/1509.03770


[SCQPT] Self-Consistent Quantum Process Tomography  
Seth T. Merkel et al.  
Phys. Rev. A 87, 062119 (2013)  
https://dx.doi.org/10.1103/PhysRevA.87.062119  
https://arxiv.org/abs/1211.0322  

## Quantum process tomography in `forest.benchmarking`

Before reading this section make sure you are familiar with the [state tomography ipython notebook](tomography_state.ipynb).

The basic workflow is:

1. Prepare a process that you wish to estiamte by specifying a pyQuil program.
2. Construct a list of input and output observables that are needed to estimate the state; we collect this into an object called an `ObservablesExperiment`.
3. Acquire the data by running the program on a QVM or QPU.
4. Apply an estimator to the data to obtain an estimate of the process.
5. Compare the estimated state to the true state by a distance measure or visualization.


Below we break these steps down in to all their ghastly glory. 

In [6]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *
qc = get_qc('2q-qvm')

### Step 1. Construct a process
Which is represented as a pyQuil `Program`

In [ ]:
qubits = [0]
process = Program(RX(np.pi, qubits[0]))
print(process)

## Construct a `TomographyExperiment` for process tomography
The `I` basis measurements are redundant, and can be grouped with other terms (see below).

In [ ]:
from forest.benchmarking.tomography import generate_process_tomography_experiment
experiment = generate_process_tomography_experiment(process, qubits)
print(experiment)

## PyQuil will run the tomography programs

In [ ]:
from forest.benchmarking.observable_estimation import estimate_observables
results = list(estimate_observables(qc, experiment))
results

## Linear Inversion Estimate

Sometimes the Linear Inversion Estimates can be unphysical. But we can use `proj_choi_to_physical` to force it to be physical.

In [5]:
from forest.benchmarking.tomography import linear_inv_process_estimate
from forest.benchmarking.superoperator_tools import proj_choi_to_physical

process_choi_est_lin_inv = linear_inv_process_estimate(results, qubits)
print(np.real_if_close(np.round(process_choi_est_lin_inv, 2)))


print('\n Project the above estimate to a physical estimate:\n')
print(np.real_if_close(np.round(proj_choi_to_physical(process_choi_est_lin_inv), 2)))

[[-0.03-0.j   -0.01-0.j   -0.01+0.j   -0.  +0.02j]
 [-0.01+0.j    1.03-0.j    1.  +0.j    0.01-0.01j]
 [-0.  -0.01j  1.  -0.j    0.97-0.j    0.02+0.02j]
 [ 0.  -0.02j  0.  +0.01j  0.02-0.02j  0.03-0.j  ]]

 Project the above estimate to a physical estimate:

[[-0.  +0.j   -0.01+0.j   -0.01+0.j   -0.  +0.01j]
 [-0.01-0.j    1.  -0.j    0.99+0.j    0.01-0.j  ]
 [-0.01-0.j    0.99-0.j    0.97+0.j    0.02+0.01j]
 [-0.  -0.01j  0.01+0.j    0.02-0.01j  0.03-0.j  ]]


## PGDB Estimate

In [ ]:
from forest.benchmarking.tomography import pgdb_process_estimate
process_choi_est = pgdb_process_estimate(results, qubits)
np.real_if_close(np.round(process_choi_est, 2))

Ideal Choi Matrix

In [ ]:
from forest.benchmarking.superoperator_tools import kraus2choi
from pyquil.gate_matrices import X as X_matrix
process_choi_ideal = kraus2choi(X_matrix)
np.real_if_close(np.round(process_choi_ideal))

## Plot Pauli Transfer Matrix of Estimate

In [ ]:
import matplotlib.pyplot as plt
from forest.benchmarking.superoperator_tools import choi2pauli_liouville
from forest.benchmarking.plotting.state_process import plot_pauli_transfer_matrix

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,5))
plot_pauli_transfer_matrix(np.real(choi2pauli_liouville(process_choi_ideal)), ax1, title='Ideal')
plot_pauli_transfer_matrix(np.real(choi2pauli_liouville(process_choi_est_lin_inv)), ax2, title='Lin Inv Estimate')
plot_pauli_transfer_matrix(np.real(choi2pauli_liouville(process_choi_est)), ax3, title='PGDB Estimate')
plt.tight_layout()

## Two qubit example - CNOT

In [ ]:
qubits = [0, 1]
process = Program(CNOT(qubits[0], qubits[1]))
experiment = generate_process_tomography_experiment(process, qubits, in_basis='sic')
print(experiment)

In [ ]:
results = list(estimate_observables(qc, experiment))
results[:10]

In [ ]:
def _print_big_matrix(mat):
    for row in mat:
        for elem in row:
            elem = np.real_if_close(np.round(elem, 3), tol=1e-1)
            if not np.isclose(elem, 0., atol=1e-2):
                print(f'{elem:.1f}', end=' ')
            else:
                print(' . ', end=' ')
        print()

In [ ]:
process_choi_est = pgdb_process_estimate(results, qubits)
_print_big_matrix(process_choi_est)

In [ ]:
from pyquil.gate_matrices import CNOT as CNOT_matrix
process_choi_ideal = kraus2choi(CNOT_matrix)
_print_big_matrix(process_choi_ideal)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
ideal_ptm = choi2pauli_liouville(process_choi_ideal)
est_ptm = choi2pauli_liouville(process_choi_est)
plot_pauli_transfer_matrix(ideal_ptm, ax1, title='Ideal')
plot_pauli_transfer_matrix(est_ptm, ax2, title='Estimate')
plt.tight_layout()